# Data Science - Aula Prática Análise Titanic 

## 1. Explorando o problema

A partir do conjunto de dados do desastre do Titanic disponível na plataforma [Kaggle](https://www.kaggle.com/c/titanic/data) e disponível nesse repositório como titanic_orig.csv, tente responder as seguintes perguntas sempre justificando as suas respostas:

1. Faça uma breve exploração dos dados, ilustrando quando possível para responder as perguntas:

- A. Existe diferença significativa entre as proporções de sobreviventes entre homens e mulheres?
- B. Existe diferença significativa entre as proporções de sobreviventes entre classes diferentes?
- C. Existe diferença significativa entre as proporções de sobreviventes entre faixas etárias diferentes? Quão mais velho você precisa ser para que você não saísse vivo do desastre?
- Obs.: Você é capaz de utilizar algum teste estatístico para dar suporte aos seus resultados?

2. Quais variáveis explicam melhor os dados? Explique quais testes e modelos foram utilizados em sua resposta.

3. Crie um modelo que defina a probabilidade de sobrevivência a partir das características de cada passageiro. Obs.: Siga uma metodologia que valide o modelo criado.

4. Bônus: Qual probabilidade de um homem solteiro de 22 anos que embarcou em Southampton sozinho na terceira classe sobreviva ao desastre?



## 2. Ferramentas e Dados

### 2.1. Instalando Bibliotecas 

In [1]:
# Instalando bibliotecas

#!pip install pandas
#!pip install scipy
#!pip install -U matplotlib
#!pip install -U seaborn
#!pip install -U scikit-learn
#!pip install sqlalchemy
#!pip install psycopg2
#!pip install pandas-profiling
#!pip install ipywidgets

### 2.2. Lendo Bibliotecas

In [1]:
#JWLS 20/07
##Bibliotecas python necessárias
#Manipulacao de dados
import pandas as pd
from sqlalchemy import create_engine

#Testes e recursos estatisticos
import numpy as np
from scipy import stats
import pandas_profiling

#Analise grafica
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
%matplotlib inline

#Modelos, metricas e Machine Learning
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.ensemble._forest import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error

### 2.3. Importando a Base de Dados

Primeiramente é nescessário conhecer o banco de dados que será utilizado para obter as respostas das perguntas realizadas. Para qualquer banco de dados é preciso destinguir os tipos de variáveis. Em geral as variáveis se dividem em qualitativas (categóricas) e quantitaivas, por sua vez variáveis qualitativas se dividem em nominais (por exemplo, sexo) e ordinais (por exemplo, escolaridade), já as quantitativas se dividem em discretas e contínuas. Também é importante verificar se todas as variáveis trazem informações que explicam o fenômeno estudado, pois em geral, esses problemas envolvem grandes quantidade de dados que exigem alto custo computacional. Em alguns casos certas variáveis podem ser combinadas gerando outras que possuem maiores informações. 

Variável      |   Definição                      | Chave
--------------|----------------------------------|----------
Survived      |	Survival	                     |0 = No, 1 = Yes
pclass	      | Ticket class	                     |1 = 1st, 2 = 2nd, 3 = 3rd
sex	          | Sex	                                  |   
Age	          | Age in years	                      |
sibsp	      | # of siblings / spouses aboard the Titanic |	
parch	      |# of parents / children aboard the Titanic  |	
ticket	      | Ticket number                               |	
fare	      | Passenger fare	|
cabin	      | Cabin number	
embarked	  | Port of Embarkation	                        |C = Cherbourg, Q = Queenstown, S = Southampton

#### 2.3.1. Arquivo CSV

In [5]:
#Importando o banco de dados de um csv
dados = pd.read_csv("dataset/titanic_orig.csv")
dados.head() #Visualizar os dados, 5 primeiras linhas  #tail

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### 2.3.2. Banco Postgres

In [6]:
# Importando Dados de um Banco Postgresql

# Configs
server = '191.233.233.76'
driver = 'postgresql'
port = '49153'
dbname = 'data_warehouse_dev'
username = 'select_python'
password = 'aiZPlsyCjGT!'

In [7]:
# String de conexão
connstring = "{0}://{1}:{2}@{3}:{4}/{5}".format(driver, username, password, server, port, dbname)

# Engine
engine = create_engine(connstring, pool_pre_ping=True)
conn = engine.connect()

- **Método read_sql_query**:
Permite fazer todo tipo de query SQL. com SELECT, WERE, GROUP BY, JOIN's, entre outras funcoes. 

Parâmetros:
   - sql = String SQL Query que será executada para retornar os dados
   - con = engine de conexão
   - index_col = coluna que será utilizada como indice de um data frame
   - params = Lista de parametros para serem passados ao método

In [8]:
pd.read_sql_query("""SELECT table_name
                      FROM information_schema.tables
                     WHERE table_schema='public'
                       AND table_type='BASE TABLE';""", con = engine)

,table_name
0,contratos_ativos
1,titanic


- **Lendo uma Tabela do Postgresql**

In [9]:
dados = pd.read_sql(sql="select * from public.titanic", con=engine)                  # Lê dados do banco postgres
dados.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [12]:
dados[['Sex', 'Name']]

,Sex,Name
0,male,"Braund, Mr. Owen Harris"
1,female,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,female,"Heikkinen, Miss. Laina"
3,female,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,male,"Allen, Mr. William Henry"
...,...,...
886,male,"Montvila, Rev. Juozas"
887,female,"Graham, Miss. Margaret Edith"
888,female,"Johnston, Miss. Catherine Helen ""Carrie"""
889,male,"Behr, Mr. Karl Howell"


#### 2.3.3.  Merge em DataFrame

<br>

- Tipos de JOINs:

    - INNER JOIN: (interseção) retorna todos os registros em que as chaves são semelhantes nos dois conjuntos
    - LEFT JOIN: (união de "a" com a parte de "b" que faz interseção com "a") Une todos os registros de "a" com apenas os registros de "b" que possuem a mesma chave dos registros de "a".
    - RIGTH JOIN: (união de "b" com a parte de "a" que faz interseção com "b") Une todos os registros de "b" com apenas os registros de "a" que possuem a mesma chave dos registros de "b".
    - OUTER JOIN: (união) une todos os registros independente de chaves.

In [13]:
# Amostra dos Dados

merge1 = dados[["PassengerId", "Survived"]].sample(20, random_state = 123)
merge1

,PassengerId,Survived
172,173,1
524,525,0
452,453,0
170,171,0
620,621,0
397,398,0
161,162,1
41,42,0
702,703,0
567,568,0


In [14]:
merge2 = dados[["PassengerId", "Sex"]].sample(20, random_state = 149)
merge2

,PassengerId,Sex
382,383,male
488,489,male
856,857,female
665,666,male
658,659,male
61,62,female
153,154,male
783,784,male
703,704,male
161,162,female


- INNER JOIN

In [15]:
inner_join = pd.merge(merge1, merge2, on = "PassengerId", how = "inner")
inner_join

,PassengerId,Survived,Sex
0,162,1,female
1,703,0,female


- LEFT JOIN

In [16]:
left_join = pd.merge(merge1, merge2, on = "PassengerId", how = "left")
left_join

,PassengerId,Survived,Sex
0,173,1,NaN
1,525,0,NaN
2,453,0,NaN
3,171,0,NaN
4,621,0,NaN
5,398,0,NaN
6,162,1,female
7,42,0,NaN
8,703,0,female
9,568,0,NaN


- RIGHT JOIN

In [17]:
right_join = pd.merge(merge1, merge2, on = "PassengerId", how = "right")
right_join

,PassengerId,Survived,Sex
0,383,NaN,male
1,489,NaN,male
2,857,NaN,female
3,666,NaN,male
4,659,NaN,male
5,62,NaN,female
6,154,NaN,male
7,784,NaN,male
8,704,NaN,male
9,162,1.0,female


- OUTER JOIN

In [18]:
outer_join = pd.merge(merge1, merge2, on = "PassengerId", how = "outer")
outer_join

,PassengerId,Survived,Sex
0,173,1.0,NaN
1,525,0.0,NaN
2,453,0.0,NaN
3,171,0.0,NaN
4,621,0.0,NaN
5,398,0.0,NaN
6,162,1.0,female
7,42,0.0,NaN
8,703,0.0,female
9,568,0.0,NaN


#### 2.3.4. Trabalhando com Agrupamentos

In [23]:
# Groupby po Sexo
dados[["Survived", "Sex"]].groupby(["Sex"], as_index=False).count()

,Sex,Survived
0,female,314
1,male,577


In [24]:
# Groupby po Sexo e Sobrevivência
dados[["Survived", "Sex", "Age", "Fare"]].groupby(["Survived", "Sex"], as_index=False).mean()

,Survived,Sex,Age,Fare
0,0,female,25.046875,23.024385
1,0,male,31.618056,21.960993
2,1,female,28.847716,51.938573
3,1,male,27.276022,40.821484


#### 2.3.5 Outros Métodos Pandas

In [6]:
# Tamanho da base de dados
dados.shape

(891, 12)

In [7]:
# Visualizando o tipo dos dados
dados.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [8]:
# Descrição dos dados
dados.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 3. Trabalhando com Valores Faltantes e Engenharia de Variáveis

### 3.1. Valores faltantes

Neste banco de dados temos variáveis categóricas e quantitativas (discretas e contínuas). As variáveis categóricas aprecem na forma numérica (int) e de texto (str), em python temos que transformar as variáveis de texto em númericas. Também é possível verificar a existência de valores faltantes 'NaN' nas variáveis Age, Cabin e Embarked.

O tratamento de valores faltantes é parte crucial para qualquer análise de dados, em geral existem diversas formas de tratá-los. A forma mais simples é substituir NaN's pela média, mediana, moda ou simplesmente (em casos extremos ou que não gerem perda de informações) a exclusão da instância. quando se exige maior precisão pode-se optar por abordagens mais complexas, com uso de modelos de regressão simples ou multivariados ou de machine learning para estimativa de valores faltantes. Antes de começar a preencher NaN's é importante verificar se essa variável é importante para o estudo e quais outras variáveis podem ser utilizadas para estimar um valor faltante. Existem diferentes formas de imputação de valores faltantes, algumas são:

- Imputação simples

Existindo poucos valores NaN's na variável estudada podemos substituí-lo utilizando alguma medida de tendência central, é uma forma mais simples e rápida.

- Imputação simples com regressão

Neste  método  são  considerados  os  valores  das  demais  características  para estimar o valor faltante. Isto pode ser feito baseado em modelos de regressão logístico (binária), multinomial ou linear.

- Imputação múltipla

A  Imputação  Múltipla  é  uma  técnica  para  analisar  bancos  de  dados onde  algumas  entradas  são  faltantes.  A  aplicação  dessa técnica  requer  três  passos:  imputação,  análise  e  agrupamento. 

O banco estudado possui valores faltantes nas variáveis Age, Cabin e Embarked. Calculando o total de NaN podemos identificar a melhor forma de preencher esses valores. Assim, calculamos a soma de valores faltantes para cada variável:

In [13]:
print('Soma de valores faltantes em Age:      ', np.mean(dados["Age"].isnull()))
print('Soma de valores faltantes em Cabin:    ', np.mean(dados["Cabin"].isnull()))
print('Soma de valores faltantes em Embarked: ', np.mean(dados["Embarked"].isnull()))

Soma de valores faltantes em Age:       0.19865319865319866
Soma de valores faltantes em Cabin:     0.7710437710437711
Soma de valores faltantes em Embarked:  0.002244668911335578


A variável Cabin é composta de mais de 50%  de valores faltantes. Estimar esses valores pode trazer um viés para futuras análies, dessa forma a melhor opção é excluir essa variável de nosso banco de dados.  

In [14]:
dados2 = dados.drop("Cabin", axis = 1)             #Excluindo a coluna Cabin
dados2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


A variável Embarked possui dois valores faltantes, uma abordagem simples que pode ser utilizada e que não cause viés nas inferências posteriores é substituir esses valores pela moda, já que estamos trabalhando com variáveis categóricas. Dessa forma, vamos encontrar a moda e substituir nos valores falatantes:  

In [19]:
# Preenche valores faltantes em Embarked
moda = dados2["Embarked"].mode()
dados2["Embarked"].fillna(moda[0], inplace = True)
print(sum(dados2['Embarked'].isnull()))

0


A variável idade apresenta 177 valores omissos. A idade é uma variável mais complexa para ser estimada e sabemos que utilizar a média ou mediana não é a melhor solução, pois, podemos gerar um viés negativo ou positivo na nossa análise. A média desses 177 valores aumentarão a frequência da classe média da idade. Uma forma mais coerente é estimar as idades faltantes com modelos de regressão simples ou múltiplos ou de machine learning. Para isso, devemos utilizar o maior número de variáveis possíveis e verificar se as mesmas contribuem para a estimativa. Antes de estimar a idade é preciso tentar extrair mais informações dos dados, pricipalmente verificar se é possível criar novas variáveis ou excluir as menos importantes. 

### 3.2. Novas variáveis

A combinação de uma ou  mais variáveis pode ajudar na estimativa de sobreviventes do Titanic. Uma variável que pode ser facilmente estimada é o tamanho da família, pois temos as variáveis SibSp que indica o número de irmãos ou esposa/esposo e Parch que indica o número de pais e ou filhos. Somando estas variáveis e acrescentando 1 "a própria pessoa", temos o tamanho da família:    

In [21]:
# Adiciona variável Tamanho da familia
dados2["Family"] = dados2["SibSp"] + dados2["Parch"] + 1                       # New variable
dados2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1


Uma observação importante é, deve-se ter cuidado ao se criar novas variáveis, pois elas podem ser autocorrelacionadas e, dessa forma, estariamos inserindo autocorrelção ao nosso modelo. Outro ponto importante é que nem sempre muitas variáveis explicam melhor os dados, e o fundamento da regressão é utilizar sempre modelos mais simples para estimativas.

A variável Name por ser categórica e cada nome representar uma categoria, não traz muita informação para estimar os sobreviventes. Entretanto, ela traz um pronome de tratamento ou grau de título, dessa forma, é importante extrair essa informação e gerar uma nova variável categórica com o título de cada pessoa.

In [22]:
#Quebra a string nome e extrai a informacao do titulo de cada pessoa
dados2["Title"] = dados2["Name"].str.extract('([A-Za-z]+)\.', expand=False)                   # Neu variable
dados2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Family,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,2,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,2,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1,Mr


Agora temos os títulos de cada pessoa, entretanto ficamos com 17 títulos diferentes. Alguns desses títulos são semelhantes ou estão na mesma classe de grau. Dessa forma, podemos reduzir o número de títulos agrupando os semelhantes na mesma categoria. 

Os títulos dados a homens nobres ou militares com algum estatuto são Capt, Col, Don, Major, Jonkheer e Sir, já mulheres nobres e de estatuto social elevado recebem títulos de Dona, Lady e Countess. Os títulos de Miss e Mlle são para mulheres solteiras, Ms designa uma mulher sem indicaçao de estado civil, porém nos 2 casos ocorridos, as mesmas viajavam sozinhas, assim foram consideradas solteiras. As mulheres casadas são chamadas de Mrs e Mme. Dessa forma, podemos agrupar alguns desses títulos com outros de maior frequência. O títulos de menor frequência e atribuídos a pessoas nobres foram substituídos por Rich. 

- **Crosstab**

Tabelada cruzada de informações por colunas desejadas. Esta função cruza as informações de uma coluna com as informações de outra coluna.

In [23]:
#Frequência de titulos
pd.crosstab(dados2["Title"], dados["Sex"])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [24]:
#Agrupando passageiros em titulos mais comuns
dados2['Title'] = dados2['Title'].replace(['Jonkheer', 'Don', 'Capt', 'Major', 'Col', 'Countess', 'Lady',
                                          'Dr', 'Rev', 'Sir'], 'Rich')
dados2['Title'] = dados2['Title'].replace(['Mlle', 'Ms'], 'Miss')
dados2['Title'] = dados2['Title'].replace(['Mme'], 'Mrs')

In [25]:
#Titulos por sexo
pd.crosstab(dados2['Title'], dados2['Sex'])

Sex,female,male
Title,,
Master,0,40
Miss,185,0
Mr,0,517
Mrs,126,0
Rich,3,20


Depois desse agrupamento ficamos com 5 categorias na variável título. Agora a variável Name não é mais importante podendo ser excluída do nosso banco de dados. Outra variável que ainda não foi tratada é a Ticket. Analisando um pouco essa variável pode-se notar que ela apresenta valores numéricos e de texto, em geral as letras podem indicar uma classe de valores dos tickets. Porém, não existe uma descrição para essa variável e as informações obtidas pela Classe social e pelo valor pago da passagem trazem bastante informação sobre as condições dos passageiros. Assim, essas variáveis podem ser excluídas do banco de dados.     

In [26]:
#Excluindo Name e Ticket
dados3 = dados2.drop(['Name', 'Ticket'], axis = 1)
dados3.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,Title
0,1,0,3,male,22.0,1,0,7.2500,S,2,Mr
1,2,1,1,female,38.0,1,0,71.2833,C,2,Mrs
2,3,1,3,female,26.0,0,0,7.9250,S,1,Miss
3,4,1,1,female,35.0,1,0,53.1000,S,2,Mrs
4,5,0,3,male,35.0,0,0,8.0500,S,1,Mr


### 3.3. Estimando Valores Faltantes

#### 3.3.1. Variáveis categóricas e contínuas

O banco de dados está quase pronto para podermos realizar nossa exploração. Sabemos que nosso banco é composto de variáveis quantitativas (contínuas e discretas) e qualitativas "ou categóricas" (nominal e ordinal). As variáveis quantitativas estão prontas para análise, porém as categóricas precisam ser ajustadas para trabalharmos com python. A melhor forma de trabalhar com essas variáveis é atribuir valores para cada atributo, com isso não teremos problemas posteriores. 

In [27]:
#modulo LabelEncoder da biblioteca sklearn
labelencoder = LabelEncoder()

dados4 = dados3
dados4['Sex'] = labelencoder.fit_transform(dados4['Sex'])
dados4['Embarked'] = labelencoder.fit_transform(dados4['Embarked'])
dados4['Title'] = labelencoder.fit_transform(dados4['Title'])
dados4.head()

#Codificacoes
#Sex
#female = 0, male = 1,
#Embarked
#C = 0, Q = 1, S = 2
#Title
#Master = 0, Miss = 1, Mr = 2, Mrs = 3, Rich = 4

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,Title
0,1,0,3,1,22.0,1,0,7.2500,2,2,2
1,2,1,1,0,38.0,1,0,71.2833,0,2,3
2,3,1,3,0,26.0,0,0,7.9250,2,1,1
3,4,1,1,0,35.0,1,0,53.1000,2,2,3
4,5,0,3,1,35.0,0,0,8.0500,2,1,2


#### 3.3.2. Regressão Múltipla (NaN's de Idade)

Com o banco de dados pronto é possível estimar os valores para idade. Seguiremos duas abordagens para identificarmos qual a mais apropriada. A primeira abordagem é ajustar um modelo linear múltiplo utilizando as demais variáveis para estimar a idade, a segunda abordagem é utilizar modelos de machine learning, Randon Forest e Máquinas de Vetores de Suporte. O modelo que obtiver melhores scores será utlizado para estimar a idade. 

In [29]:
#Gerando um banco de dados para idade

dados_idade = dados4[['Age', 'Pclass', 'Title', 'Embarked', 'Sex', 'SibSp', 'Parch', 'Fare', 'Family' ]]

#Queremos estimar a idade para isso vamos usar apenas as intâncias com as idades completas
#Divindo o banco em idade_conhecida e idade_desconhecida

idade_conh = dados_idade.loc[(dados_idade.Age.notnull())]                      # Dados com idade 
idade_desc = dados_idade.loc[(dados_idade.Age.isnull())]                      # Dados sem idade
        
#Definindo a variável resposta (dependente) e as variáveis preditoras (independentes)
# dependente, resposta, target
y = idade_conh.values[:, 0]

# independente, explicativa
X = idade_conh.values[:, 1::]


#Preparando as variáveis para estimar os valor de idade perdidas
X_p = idade_desc.values[:, 1::]


Na existência de variáveis categóricas, geralmente substituímos as categorias por números inteiros, porém em modelos de machine learning, isso pode causar um viés, pois o algoritmo pode considerar que existe uma relação de ordem na variável. Uma forma de corrigir esse problema é com uso de variáveis dummy, esse procedimento conciste em criar um vetor para cada fator e adicionar um valor binário 0 ou 1, para a ocorrência do fator, logo uma variável dummy com 3 fatores será substituída por 3 vetores. Outra forma de diminuir o ruído dos dados é normalizar as variáveis, isso diminui a variância dos dados permitindo melhores ajustes.  

In [30]:
#Normalizando os dados
sc = StandardScaler()
X_treino = sc.fit_transform(X)
X_est = sc.transform(X_p)


In [ ]:
#Definindo linear multiplo


#Ajustando o modelo
                                                          # Ajusta o modelo

print(r2_score(y, modelo_linear.predict(X_treino)))       # R2 Score   
mean_squared_error(y, modelo_linear.predict(X_treino))    #Erro quadratico medio

In [ ]:
#Definindo o modelo Random Forest



print(r2_score(y, modelo_rf.predict(X_treino)))
mean_squared_error(y, modelo_rf.predict(X_treino))

In [ ]:
#Definindo o modelo Support Verctor Regression




print(r2_score(y, modelo_svr.predict(X_treino)))
mean_squared_error(y, modelo_svr.predict(X_treino))

O modelo que apresentou melhores resultados foi o Random Forest. Vamos preencher o valores faltantes com o resultado do melhor modelo. 

In [ ]:
idades_pred = modelo_rf.predict(X_est)

#Substituindo no banco de dados
dados4.loc[(dados4.Age.isnull()), 'Age'] = idades_pred

Agora vamos verificar se os valores estimados se distribuem próximo dos valores reais e assim podemos verifar se o modelo é o ideal para preencher valores faltantes na idade. Podemos visualizar isso pelo gráfico de histograma.

In [ ]:
# Grafico com idade Real e Estimada


## 4.  Estatística Descritiva

Como o banco de dados completo, podemos fazer diversas inferências sobre o incidente com o Titanic. Uma análise descritiva é primordial para nos dar conhecimento prévio sobre a distribuição dos dados. Nesta análise utilizaremos medidas de resumo e gráficos, também começaremos a responder as perguntas do teste.

Primeiro vamos verificar o número de passageiros por sexo e classe social.

In [ ]:
#Palette de cores para os graficos
tabela_cores = ['#78C850',  # Grass
                 '#F08030',  # Fire
                 '#6890F0',  # Water
                 '#A8B820',  # Bug
                 '#A8A878',  # Normal
                 '#A040A0',  # Poison
                 '#F8D030',  # Electric
                 '#E0C068',  # Ground
                 '#EE99AC',  # Fairy
                 '#C03028',  # Fighting
                 '#F85888',  # Psychic
                 '#B8A038',  # Rock
                 '#705898',  # Ghost
                 '#98D8D8',  # Ice
                 '#7038F8',  # Dragon
                ]

### 4.1. Usando o Profiling

In [ ]:
pandas_profiling.ProfileReport(dados3)

### 4.2. Matplotlib e Seaborn

In [ ]:
#Graficos de numero de passageiros por sexo e classe
plt.subplots(figsize=([17,6]))
plt.subplot(121)
ax = sns.barplot(x=dados4['Sex'], y=dados4.Sex.value_counts(), ci = None, palette = tabela_cores)
ax.set_xticklabels(labels = ['Masculino', 'Feminino'])
ax.set(xlabel='', ylabel='Número de passageiros no Titanic')
plt.subplot(122)
ax = plt.bar([2, 0, 1], dados4.Pclass.value_counts(sort = False), color = tabela_cores)
plt.xticks([0, 1, 2], ('1° Classe', '2° Classe', '3° Classe'))
plt.ylabel('Número de passageiros no Titanic')
sns.despine()

É importante verificar a distribuição da idade dos passageiros. 

In [ ]:
#Graficos da distribuicao de passageiros por idades

plt.figure(figsize=(10,7))
ax = sns.histplot(dados4['Age'], kde = False, color = 'blue', bins = 15)
ax.set(xlabel='Idade', ylabel = 'Número de passageiros no Titanic')
plt.show()

## 5. Respondendo as Questões
 
Agora vamos começar a responder as questões do teste.

1) Existe diferença significativa entre as proporções de sobreviventes entre homens e mulheres?

#### Resposta:

Primeiro calculamos as proporções de homens e mulheres sobreviventes. Essa proporção é dada na tabela abaixo.

In [ ]:
#Sobreviventes por sexo
dados4[['Sex','Survived']].groupby(['Sex'], as_index = False).mean()

O número de mulheres que sobreviveram é acentuadamente maior em relação aos homens. Podemos avaliar a associação existente entre variáveis qualitativas realizando o teste $\chi^2$ "qui-quadrado". O princípio básico deste teste é comparar proporções, ou seja, as possíveis divergências entre as frequências observadas e esperadas para um evento. 
A hipóteses que queremos testar são:

- **Hipótese nula:** As frequências observadas não são diferentes das frequências esperadas. Não existe diferença entre as frequências (contagens) de sobreviventes por sexo. Assim, não existe associação entre os grupos, sobreviventes por sexo.

- **Hipótese alternativa:** As frequências observadas são diferentes das frequências esperadas. Portanto existe diferença entre as frequências. Assim, existe associação entre os grupos sobreviventes por sexo.


#### Pivot Tables

São responsáveis por fazer agrupamento dos dados com base em colunas epecifícas. Em python use a função pivot_table, com os seguintes argumentos:

- coluna que será realizado os cálculos, pode ser soma, média, std, etc;
- index: lista de colunas para agregação
- aggfunc: define a função de agregação empregada, média, median, etc;
- margins: se True calcula a quantidade total no final da tabela.

Para calcularmos o teste $\chi^2$ criamos uma tabela com as frequências de sobreviventes por sexo: 

In [ ]:
#Tabela de contigencia




In [ ]:
#Teste X2 qui-quadrado sobreviventes por sexo
obs = [[tabela_sbys[0],tabela_sbys[1]]]
x2, p, dof, exp = stats.chi2_contingency(obs)
x2, p, dof    #X2 calculado, p_valor, graus de liberdade

O valor do $\chi^2$ foi de 260,72 com p-valor de 1,19$e^{-58}$, portanto, existe diferenças significativas entre os sobreviventes por sexo, havendo influência do sexo em relação a sobrevivencia.

2) Existe diferença significativa entre as proporções de sobreviventes entre classes diferentes?

#### Resposta:

O raciocínio para resolução dessa questão é o mesmo da quetão anterior, calculamos as proporções sobreviventes por classe social. Em seguida faremos uma tabela de contigência e faremos o teste qui-quadrado.

In [ ]:
#Sobreviventes por classe social
dados4[['Pclass','Survived']].groupby(['Pclass'], as_index = False).mean()

In [ ]:
#Tabela de contigencia
tabela_sbyc = pd.pivot_table(dados4, values = 'N', index = ['Pclass'], columns ='Survived', aggfunc = np.sum)
tabela_sbyc

In [ ]:
#Teste X2 qui-quadrado sobreviventes por classe
obs_c = [[tabela_sbyc[0],tabela_sbyc[1]]]
x2_c, p_c, dof_c, exp_c = stats.chi2_contingency(obs_c)
x2_c, p_c, dof_c    #X2 calculado, p_valor, graus de liberdade

O valor do $\chi^2$ foi de 102,89 com p-valor de 4,24$e^{-23}$, portanto, existe diferenças significativas entre os sobreviventes por classe social, existindo influência da classe social em relação a sobrevivencia.

Para uma interpretação mais interativa foi criado gráficos com relação aos sobreviventes por sexo, classe social e por sexo e classe social.

In [ ]:
#Graficos de sobreviventes por sexo e classe social

plt.subplots(figsize=([17,5]))
plt.subplot(131)
ax = sns.barplot(x = 'Sex', y ='Survived', data = dados, ci = None, palette = tabela_cores)
ax.set_xticklabels(labels = ['Masculino', 'Feminino'])
ax.set(xlabel='', ylabel='Proporção de Sobreviventes')
sns.despine()
plt.subplot(132)
ax = sns.barplot(x = 'Pclass', y='Survived', data = dados, ci = None, palette = tabela_cores)
ax.set_xticklabels(labels = ['1° Classe', '2° Classe', '3° Classe'])
ax.set(xlabel='', ylabel='Proporção de Sobreviventes')
sns.despine()
plt.subplot(133)
ax = sns.barplot(x='Pclass', y='Survived', hue='Sex', data = dados, ci =None, palette = tabela_cores)
line1 = mlines.Line2D([], [], color='#78C850', label='Masculino', linewidth=3)
line2 = mlines.Line2D([], [], color='#F08030',  label='Feminino', linewidth=3)
plt.legend(ncol=1, loc="best", handles=[line1, line2])
ax.set_xticklabels(labels = ['1° Classe', '2° Classe', '3° Classe'])
ax.set(xlabel ='', ylabel='Proporção de Sobreviventes')
sns.despine()
plt.show()


Pelo gráfico de sobreviventes por sexo, temos que mais de 74,20% das mulheres sobreviveram enquanto apenas 18,89% dos homens sobrevivrem, pelo teste qui-quadrado e considerando um nível de significância de 5% (p-valor) essa diferença é . Em relação a classe social, 62,96% dos passageiros da primeira classe sobreviveram enquanto que apenas 24,23% dos passageiros da 3° classe sobreviveram, é importante ressaltar que a 3° classe possui o maior número de passageiros. De forma geral as mulheres possuem a maior chance de sobreviver independente da classe, no entanto mulheres da primeira classe possuem o dobro de chances de sobreviverem em relação à mulheres da 3° classe. Já, homens possuem menor chance de sobrevivência em relação às mulheres, independente da classe. No entanto, homens da 1° classe duas vezes mais chances de sobreviverem que homens da 3° classe.     

3) Existe diferença significativa entre as proporções de sobreviventes entre faixas etárias diferentes? Quão mais velho você precisa ser para que você não saísse vivo do desastre?

#### Resposta

A pirâmide etária definida pelo IBGE possui 21 classes, porém com esse número de classes fica difícil visulaizar alguma informção de forma mais simples. Segundo os estatísticos Moretin e Bussab o mínimo de 5 e o máximo 15 classes é o mais indicado para o resumo de qualquer variável. Dessa forma, utilizaremos 11 classes para resumir a faixa etária de passageiros. Posteriomente, criamos a sequência com os intervalos das classes e em seguida vamos criar uma nova coluna na nossa tabela com a indicação da faxa etária de cada passageiro. As classes representam os intervalos numéricos em que a variável quantitativa foi classificada. A amplitude da classe é determinada por $\frac{max(Idade) - min(Idade)}{N. classes}$.


In [ ]:
#Definindo a sequencia de classes, a menor idade é 0,4 e a maior é 81 anos,
#assim,o valores minimo e maximo do nosso intervalo sera 0 e 81.
#Amplitude

n = 10
amp = round((dados4['Age'].max() - dados4['Age'].min())/n)
intervalos = list(range(0, 89, int(amp)))

#Agrupando a idade pela faixa etaria
dados4['FaixaEtaria'] = np.nan

for i in range(len(intervalos)-1):
    dados4.loc[(dados4['Age'] >= intervalos[i]) & (dados4['Age'] < intervalos[i+1]), 'FaixaEtaria'] = i  

In [ ]:
#Sobreviventes por faixa etária
dados4[['FaixaEtaria','Survived']].groupby(['FaixaEtaria'], as_index = False).mean()

In [ ]:
#Graficos da ddensidade de sobreviventes por faixa etaria

plt.figure(figsize=(10,7))
ax = sns.barplot(x='FaixaEtaria', y='Survived', data = dados4, ci = None, palette = tabela_cores)
ax.set_xticklabels(labels = ['0 a 7', '8 a 15', '16 a 23', '24 a 31', '32 a 39', '40 a 47',
                            '48 a 55', '56 a 63', '64 a 71', '72 a 79', '80 a 87'])
ax.set(xlabel='Idades', ylabel='Proporção de Sobreviventes')
sns.despine()
plt.show()

Vamos verificar se existe diferenças entre as faixas etárias, criamos uma tabela de contigência com as frequências de sobreviventes e não sobreviventes, poteriomente será realizado o teste $\chi^2$.

In [ ]:
#Tabela de contigencia
tabela_sbyf = pd.pivot_table(dados4, values = 'N', index = ['FaixaEtaria'], columns ='Survived', aggfunc = np.sum)

tabela_sbyf[0].fillna(0, inplace = True)
tabela_sbyf[1].fillna(0, inplace = True)
tabela_sbyf

In [ ]:
#Teste X2 qui-quadrado sobreviventes por faixa etaria
obs_fe = [[tabela_sbyf[0],tabela_sbyf[1]]]
x2_fe, p_fe, dof_fe, exp_fe = stats.chi2_contingency(obs_fe)
x2_fe, p_fe, dof_fe    #X2 calculado, p_valor, graus de liberdade

Dessa forma, verificamos que existe diferença significativa entre o grupo de sobreviventes pelas diferentes faixas etárias, ou seja, a faixa etária influencia nas chances de sobrevivência. Em relação há quanto mais velho você precisa ser para ter menos chances de sobreviver,  é possível responder esta pergunta pelo gráfico da proporção de sobreviventes por faixa etária. Por esse gráfico podemos visualizar que a maior chance de sobreviver é de passageiros na classe de 0 a 7 anos, logo a partir do limite dessa classe as chances de sobreviver diminuem, cabe resaltar que a partir dos 64 anos de vida a probabilidade de sobrevivência foi quase nula, há não ser pelo fato que na classe de 80 a 88 anos, 100% dos passageiros sobreviveram, porém pela tabela de contigência constatamos que existe apenas uma ocorrência para essa faixa etária, ou seja, apenas uma pessoa acima de 64 anos de idade sobreviveu. Logo, qualquer pessoa com menos de 64 anos de idade possui melhores chances de sobreviver e com menos de 7 anos de idade essa probabilidade aumenta para mais de 60%.  

4) Quais variáveis explicam melhor os dados? Explique quais testes e modelos foram utilizados em sua resposta.

#### Resposta:

 Em um primeiro instante, pode-se pensar que quanto maior o banco de dados, representados por um volume elevado de variáveis descritivas de observações,  seja  preferível  para  a  explicação de um fenômeno.  Contudo,  algumas variáveis não acrescentam nehuma informação adicional para explicação do fenômeno, além de um grande número de variáveis aumentar a dimensionalidade dos dados gerando alto custo computacional e as vezes inserindo um viés ao modelo. A alta dimensionalidade, ito é, muitos atributos, ou colunas falando de banco de dados é um fator crítico para o desempenho de muitos algoritmos.  Diasnte disso, é importante verificar quais as variáveis que mais contribuem para o estudo. A correlação entre as variáveis dependentes pode trazer essa informação, lembrando que, variáveis que apresentarem baixa correlação podem ser excluídas do modelo e variáveis com alta correlação entre si podem ser substituidas uma pela outra para evitar o problema da autocorrelação. Cabe ressaltar que quase todos os métodos da estatística classe para inferir contribuição de variáveis são lineares, logo ao se trabalhar com problemas não lineares esses métodos apresentam baixa eficiência. Uma alternativa é fazer uso de modelos de machine learning como, árvores de decisões, florestas aleátorias e redes neurais.

In [ ]:
#Dados
dados5 = dados4[['Survived', 'Pclass', 'Sex','Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 
                 'Family', 'Title', 'FaixaEtaria']]

#Definindo a variavel resposta (dependente) e as variáveis preditoras (independentes)

y_target = dados5.values[:, 0]
X_feature = dados5.values[:, 1::]


In [ ]:
#Matriz de correlacao

matriz_corr = dados5.corr(method="spearman")
ax = plt.subplots(figsize=(25,16))
sns.set(font_scale = 1.5)
sns.heatmap(matriz_corr, cmap='viridis', linewidths=0.1,vmax=1.0, square=True, annot=True)
plt.show()


A Matriz de Correlação permite calcular a correlação entre variáveis através dos coeficiente de correlção de Pearson, Sperman ou Kendal. O heatmap criado acima, indica o nível de correlação de cada variável em uma graduação de cores, 
quanto mais amarela a variável, maior a correlação. Esse gráfico é útil para detectar quais variáveis possuem maior correlação com a variável respota e com as demais variáveis independentes. Pelo gráfico as variáveis que possuem maior correlação com a variável resposta são: Classe, Sexo, Fare e Embarked, logo essas variáveis podem explicar melhor a sobrevivência. Já, as variáveis Age e FaixaEtaria são autocorrelacionadas, bem como as variáveis Family, SibSp e Parch. Em modelos de regressão essas variáveis devem ser substituídas pela que apresentar maior correlação com a variável sobrevivência. 

A matriz de correlação é um método linear, diante disso em problemas não lineares ela pode não encontrar relção entre as variáveis. Para reverter o problema da não linearidade foi utilizado o modelo Random Forest para estimar a sobrevivência e classificar as variáveis mais importantes. As metodologias baseadas em árvores de decisão são as mais utilizadas para encontrar relação entre variáveis.

In [ ]:
#Definindo o modelo Random Forest

modelo_rf1 = RandomForestRegressor(n_estimators=1000)
modelo_rf1.fit(X_feature, y_target)
print(r2_score(y_target, modelo_rf1.predict(X_feature)))
mean_squared_error(y_target, modelo_rf1.predict(X_feature))

# Importância das variáveis
importances = modelo_rf1.feature_importances_
std = np.std([tree.feature_importances_ for tree in modelo_rf1.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
# Grafico para importancia das variaveis
plt.figure(figsize = ([15,8]))
plt.bar(range(X_feature.shape[1]), importances[indices],
       color=tabela_cores, align="center")
plt.xticks(range(X_feature.shape[1]), ['Sexo', 'Idade', 'Taxa', 'Classe', 'Família', 'Faixa Etária', 
                                       'Irmãos/Conjugues', 'Título',  'Porto', 'Filhos/Pais'])
plt.xlim([-1, X_feature.shape[1]])
plt.show()

#['Sexo', 'Idade', 'Taxa', 'Classe', 'Família', 'Faixa Etária', 'Irmãos/Conjugues', 'Título',  'Porto', 'Filhos/Pais']
#['Pclass', 'Sex','Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Family', 'Title', 'FaixaEtaria']

Com o modelo random forest classificamos as variáveis mais importantes. Assim, Sexo, Idade, Taxa e Classe são as variáveis que possuem maior informção sobre a variável sobrevivência. É importante destacar que, se as variáveis independentes não são autocorrelacionas "o que gera estimativa viesada em modelos de regressão", podemos utilizar todas as variáveis ou apenas as mais importantes que os resultados são semelhantes. Usar todas as variáveis, salvo a única restrição de autocorrelação em modelos de regressão, não prejudica significativamente o desempenho do modelo, apenas gera maior custo computacional, se o custo computacional é um fator limitante, estimativas com as variáveis mais importantes gera resultados estatísticamente semelhantes.  

3) Crie um modelo que defina a probabilidade de sobrevivência a partir das características de cada passageiro. Obs.: Siga uma metodologia que valide o modelo criado.

#### Resposta:

O modelo clássico para este tipo de análise é o modelo de regressão logística. Com ele podemos estimar a probalidade de alguém sobreviver de acordo com as informações obtidas com as demais variáveis. 

In [ ]:
#Dados

#Todas as variaveis
dados5 = dados4[['Survived', 'Pclass', 'Sex','Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 
                 'Family', 'Title', 'FaixaEtaria']]


#Variaveis mais importantes pelo random forest
dados6 = dados4[['Survived', 'Pclass', 'Sex','Age', 'Fare']]


#Todas as variaveis

y_target = dados5.values[:, 0]
X_feature = dados5.values[:, 1::]

#Variaveis escolhida pelo Random forest

y_target1 = dados6.values[:, 0]
X_feature1 = dados6.values[:, 1::]


In [ ]:
#Regressao logistica com todas as variaveis

modelo_lr = LogisticRegression(C=1., solver='lbfgs', max_iter=999)
modelo_lr.fit(X_feature, y_target)

# Qualidade do modelos ajustado
print(modelo_lr.score(X_feature, y_target))
mean_squared_error(y_target, modelo_lr.predict(X_feature))

In [ ]:
#Regressao logistica com as variaveis Sexo, Idade, Classe e Taxa

modelo_lr1 = LogisticRegression(C=1., solver='lbfgs')
modelo_lr1.fit(X_feature1, y_target1)

# Qualidade do modelo ajustado
print(modelo_lr1.score(X_feature1, y_target1))        
mean_squared_error(y_target1, modelo_lr1.predict(X_feature1))

O resultados com ambos os modelos logísticos são semelhantes, daí podemos concluir que com apenas as quatros variáveis podemos obter resultados semelhantes e no caso desse modelo de regressão não caímos no problema de autoccorrelação de variáveis. A acurácia do modelo é de aproximadamente 80%, ou seja, o modelo está bem ajustado.

Como não exite preocupação com o custo computacional podemos treinar o algoritmo random forest com todas as variáveis e verificar o seu desenpenho.

In [ ]:
modelo_rf2 = RandomForestRegressor(n_estimators=1000)
modelo_rf2.fit(X_feature1, y_target1)

# Qualidade do modelo ajustado
print(r2_score(y_target1, modelo_rf2.predict(X_feature1)))
mean_squared_error(y_target1, modelo_rf2.predict(X_feature1))


Todos os modelos apresentam resultados estatisticamente significantes, porém o modelo com menor erro quadrado médio é o mais indicado para realizar estimativas. Um ponto positivo do modelo logístico é que é possível facilmente estimar a probabilidade de sobrevivência de um passageiro.

4) Bônus: Qual probabilidade de um homem solteiro de 22 anos que embarcou em Southampton sozinho na terceira classe sobreviva ao desastre?

#### Resposta:


In [ ]:
# Calculando a probalidade

# Variáveis

# Solteiro: 

# Age: 22
# Sex: male = 1
# Embarked: C = 0, Q = 1, S = 2
#Title
#Master = 0, Miss = 1, Mr = 2, Mrs = 3, Rich = 4
#['Pclass', 'Sex','Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Family', 'Title', 'FaixaEtaria']

X_novo = np.array([[3., 1., 22., 0.],])
modelo_lr1.predict_proba(X_novo)



Logo, um passageiro com essas características tem probalidade de sobrevivência de xx%.